Bare-bones example to run semi-grand canonical MC simulations using user-specified GC steps. Note that `Semigrandcanonicaltableswapper` is very similar to `Tableswapper` written for canonical disordering across multiple sublattices (see `running-canonical-mc-with-overlapping-sublattices.ipynb`, J. Yang and T. Chen for a tutorial for `Tableswapper`).

Author: Julia Yang

May 2022

## 1. Load the CE, starting spinel structure, and supercell matrix for semi-grand canonical (SGC) swaps

In [1]:
import json
import numpy as np
from smol.io import load_work
from smol.cofe.expansion import ClusterExpansion 

starting_spinel_file = 'data/limn2o4_spinel.json'

mson = 'data/lmof_drx_tet.mson'

# load the starting configuration which is a spinel structure
from pymatgen.core.structure import *
with open(starting_spinel_file, 'r') as f: 
    init_struct = Structure.from_dict(json.load(f))

# this is the supercell shape of the spinel structure
m = np.matrix([[2,2,-2],[-2,2,2],[2,-2,2]]) 

# run a small cell for now 
transformation = np.identity(3)
supercell_matrix = m @ transformation

# load CE
work = load_work(mson)
expansion = work['ClusterExpansion']

## 2. Initialize semi-grand canonical information 

High-level description: In running a topotactic delithiation, we need to specify the kinds of semi-grand canonical swaps expected. In this case, a single swap proposal is the removal of Li+ and oxidation of Mn, or a Li+ insertion and reduction of Mn. Specifying these swaps is done with `GC_flip_table`. We also would like Li+ to disorder across multiple sublattices, but only Li+, which is why we cannot just indicate `allow_crossover=True` since that would mobilize the Mn2+ (across tetrahedral and octahedral shared sublattices). Therefore, we must specify a more specific swap, which is the shared sublattice swap with Li+ and Vacancy, and we do that clearly in `shared_table`. Technically `allow_crossover` is still `True` which is why it's kept as input. In the example below, we do SGC swaps for 50% of the proposals, and shared Li/Vacancy canonical swaps for the other 50%.

In summary, for clarity, and for ease of integration, we keep the logic within `Semigrandcanonicaltableswapper` very similar to that of `Tableswapper`. 

#####  Setting up `GC_flip_table`: 

Here we allow for the two (de)lithiation reactions with probability equal to 0.5: 

1. Li+ + Mn2+ <-(>) Vac + Mn3+ 
2. Li+ + Mn3+ <-(>) Vac + Mn4+ 

Currently as implemented, if you specify Vacancy, you must specify this as `str(Vac)`. This is converted into a smol Vacancy (DummySpecie) later by the function `make_shared_swap_table`. 

##### Initialize relative chemical potentials: 

We also need to initiate the chemical potentials for all the species. Note that it's always the relative chemical potential that matters in a SGC calculation, so these can be set arbitrarily. Just be cognizant of what you are enforcing when you scan in different chemical potentials. If you aren't seeing anything happen, you may need to scan in a different chemical potential range, or change the reference chemical potentials. 

In [2]:
# Define sgc steps 

GC_flip_table = {(
            (('Li', 1), ('Mn', 2)),
            (('Vac', 0), ('Mn', 3)))
        : 1 / 2,
        (
            (('Li', 1), ('Mn', 3)),
            (('Vac', 0), ('Mn', 4)))
        : 1 / 2,
        'Mn_disproportionation': 0.
    }

# Define swap table which only allowed for topotactic (de)lithiation on shared sublattices 
shared_table = {((('Li', 1), 'shared'), (('Vac', 0), 'shared')):1.0}

lmof_chemical_potentials = {'Li+': 0.0, 'Vacancy': 0.0, 'Mn2+': 0.0, 'Mn3+': 0.0, 'Mn4+': 0.0, 'O2-': 0.0, 'F-': 0.0}

## 3. Carry out SGC calculation

We set up a room-temperature SGC topotactic lithiation by scanning from low to high Li chemical potential. We enforce 50% of the swaps to be SGC steps in `GC_flip_table` and the other 50% to be canonical Li/Vacancy shared swaps. The parameter `combine_domains` is only for SGC flips and generally `False` so that picking of one site within a swap occurs within the same sublattice. You will generally want to keep this to `False`. (The reason it is still a parameter is that the concept of `combine_domains` is necessary in Mn disportionation reactions to satisfy detailed balance. Furthermore, maybe in some systems, one would want to perform SGC swaps across all possible cation sublattices.)

Note this bare-bones example is missing equilibration, production, and sampling steps. 

In [3]:
from smol.moca.ensemble.semigrand import SemiGrandEnsemble, make_sgc_step_table, make_shared_swap_table
from smol.moca import Sampler 

ensemble = SemiGrandEnsemble.from_cluster_expansion(expansion,
                                                    supercell_matrix,
                                                    chemical_potentials=lmof_chemical_potentials)
current_occupancy = ensemble.processor.occupancy_from_structure(init_struct)

ROOM_TEMPERATURE = 300
START_MU = -5
END_MU = -3 
STEP_MU = 0.05
STEP_TYPE = 'Semigrandcanonicaltableswapper'
NUMBER_OF_PROPOSALS=20 # quick mu scan parameter 

GC_flip_table = make_sgc_step_table(GC_flip_table)

for dmu in np.arange(START_MU, END_MU, STEP_MU):
    lmof_chemical_potentials[Species('Li', 1)] = dmu
    ensemble._mu_table = ensemble._build_mu_table(ensemble.chemical_potentials)
    sampler = Sampler.from_ensemble(kernel_type='Metropolis',
                                ensemble=ensemble,
                                temperature=ROOM_TEMPERATURE,
                                step_type=STEP_TYPE,
                                allow_crossover=True,
                                GC_step_probability=0.5,
                                swap_table=make_shared_swap_table(shared_table),
                                gc_step_table=GC_flip_table,
                                combine_domains=False,
                                )
    sampler.run(NUMBER_OF_PROPOSALS, 
               initial_occupancies=current_occupancy, 
               ignore_empty_proposals=True)


/Users/jhyang/miniconda3/lib/python3.8/site-packages/smol/moca/sampler/mcusher.py:695: UserWarning: At least one species does not exist in given sublattice in list of possible flip types (list of species on sublattice is empty). Continuing, returning an empty flip
  warnings.warn("At least one species does not exist in given "


In [4]:
save_dict = {'features': sampler.samples.get_feature_vectors().tolist(),
             'occupancies': sampler.samples.get_occupancies().tolist(),
             'efficiency': sampler.samples.sampling_efficiency}
#with open('semigrandcanonical_mc_overlapping_sublattices_example.json', 'w') as f: 
#    json.dump(save_dict, f)